<a href="https://colab.research.google.com/github/joSanchez28/BERT_on_tweets/blob/master/Libreta6_Uso_de_BERT_en_Twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Clasificación de sentimientos en Twitter

En esta libreta usaremos el mejor modelo que hemos obtenido en las libretas anteriores (BERT ajustado a nuestro conjunto de datos de Twitter) y lo usaremos para realizar clasificación de sentimientos en tweets. Para descargar los tweets, usaremos la API de Twitter llamada [tweepy](https://www.tweepy.org/).


Cargamos en primer lugar los paquetes necesarios.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!cp "/content/drive/My Drive/Colab Notebooks/lectura_y_preprocesado.py" .

In [3]:
pip install transformers

In [4]:
pip install jsonpickle

In [5]:
import os
import pandas as pd
import re
import itertools
import time
import pickle
import numpy as np
import sklearn
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
#import scikitplot as skplt
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf
from lectura_y_preprocesado import *

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


## Carga y preparación del modelo 
 En este apartado cargaremos el modelo BERT como ya hicimos. Esta vez lo prepararemos para que solo clasifique como tweets positivos/negativos aquellos a los que les asigne una etiqueta muy cercana al 1 (positivo) o al 0 (negativo). De esta forma, se considerarán neutros aquellos tweets que tengan un valor alrededor de $0.5$. Es obvio que esta forma de clasificar tweets en 3 clases (negativo, neutro o positivo) no es la idónea, pues idealmente el modelo debería de haber sido entrenado con las 3 clases. Esto no ha sido posible debido a que el conjunto de entrenamiento con el que estamos trabajando no posee tweets de clase neutra.


Cargamos el modelo BERT ya entrenado con tweets.

In [6]:
models_dir = "/content/drive/My Drive/Modelos_finales/" #Para ejecutarlo en Google Colab con Google Drive
#models_dir = "../Modelos_finales" #Para ejecutarlo localmente
model_name = "my_best_model_BERT.02-0.86.h5"
model_path = models_dir + model_name

In [7]:
model_BERT, tokenizer, config = carga_modelo_BERT(model_path)

In [8]:
pipeline_BERT = TextClassificationPipeline(model = model_BERT, tokenizer = tokenizer, framework= "tf")

In [9]:
decode_labels_map = {"LABEL_0": 0, "LABEL_1": 1}
def decode_labels(label):
    return decode_labels_map[label]

In [10]:
pipeline_BERT("I love the music.")

[{'label': 'LABEL_1', 'score': 0.9910827875137329}]

Antes de comenzar a trabajar con tweets, hagamos una pequeña prueba para ver cómo trabajaremos con las salidas del modelo.

In [11]:
predictions = pipeline_BERT("I really think Android smartphones have become the best option.", 
                            "I really think Android smartphones havent become the best option.",
                            "Leonardo seems to have lack of confidence in this movie.",
                            "I dont think the managers of this company have any idea of what the market is asking them.",
                            "Dont you think this film is awesome?",
                            "I don't like Apple products.")

In [12]:
predictions_df = pd.DataFrame(predictions)
predictions_df.label = predictions_df.label.apply(lambda x: decode_labels(x))

In [13]:
predictions_df

,label,score
0,1,0.974853
1,0,0.973031
2,0,0.863371
3,0,0.755670
4,1,0.987460
5,0,0.991117


## Uso de tweepy (API de Twitter)

Nos centramos ahora en descargar algunos tweets con la API de Twitter para posteriormente clasificarlos con BERT según su sentimiento.

In [14]:
import tweepy
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import jsonpickle

In [15]:
consumer_key = "EqeQ6v3LF0joV9grFXk4x19TJ"
consumer_secret = "FMfzwouDObjcgzZxmmFzGK6dxQIhEX4JREweiMINAkGrlNIBuj"
access_token = "1403420936-IpL9AyeoYnDLMT18ooADXl7umBKAMKs8NZmiFOi"
access_token_secret = "nSPght3oQqudUQ1BxtnEF2UmflxR95v5QXzvpPKGUJocY"

In [16]:
auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
auth.secure = True
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

Hagamos una primera query de prueba.

In [17]:
searchQuery = '@realDonaldTrump'

In [18]:
tweets = api.search(q=searchQuery, count=100, tweet_mode = 'extended')

In [19]:
#Ajuste de pandas para mostrar todo el texto de los tweets:
#pd.set_option('display.max_colwidth', None)

In [20]:
tweets_df = pd.DataFrame([tweet._json for tweet in tweets])

In [21]:
columns = list(tweets_df.columns)
print(columns)

['created_at', 'id', 'id_str', 'full_text', 'truncated', 'display_text_range', 'entities', 'metadata', 'source', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place', 'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count', 'favorite_count', 'favorited', 'retweeted', 'lang', 'quoted_status_id', 'quoted_status_id_str', 'quoted_status', 'possibly_sensitive', 'extended_entities']


In [22]:
tweets_df.shape

(100, 31)

In [23]:
tweets_df.head(3)

,created_at,id,id_str,full_text,truncated,display_text_range,entities,metadata,source,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,in_reply_to_screen_name,user,geo,coordinates,place,contributors,retweeted_status,is_quote_status,retweet_count,favorite_count,favorited,retweeted,lang,quoted_status_id,quoted_status_id_str,quoted_status,possibly_sensitive,extended_entities
0,Fri Jun 19 15:16:45 +0000 2020,1273998158985605123,1273998158985605123,RT @DineshDSouza: If @realDonaldTrump takes 20...,False,"[0, 140]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",NaN,None,NaN,None,None,"{'id': 823996821336969216, 'id_str': '82399682...",None,None,None,None,{'created_at': 'Fri Jun 19 13:04:23 +0000 2020...,False,4182,0,False,False,en,NaN,NaN,NaN,NaN,NaN
1,Fri Jun 19 15:16:45 +0000 2020,1273998158922764290,1273998158922764290,@realDonaldTrump Will someone be kneeling on t...,False,"[17, 71]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",1.273972e+18,1273972301156016130,25073877.0,25073877,realDonaldTrump,"{'id': 3065804686, 'id_str': '3065804686', 'na...",None,None,None,None,NaN,False,0,0,False,False,en,NaN,NaN,NaN,NaN,NaN
2,Fri Jun 19 15:16:45 +0000 2020,1273998158880743425,1273998158880743425,Sounds like somebody is scared with out his bu...,False,"[0, 83]","{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""https://mobile.twitter.com"" rel=""nofo...",NaN,None,NaN,None,None,"{'id': 37221831, 'id_str': '37221831', 'name':...",None,None,None,None,NaN,True,0,0,False,False,en,1.273972e+18,1273972301156016130,{'created_at': 'Fri Jun 19 13:34:00 +0000 2020...,False,NaN


La API de twitter tan solo nos deja extraer 100 tweets por consulta. Para poder extraer más tweets de forma automática, definimos ahora una función que obtenga, a través de varias consultas, un determinado número de tweets (los más recientes) que contengan un determinado término de búsqueda.


In [24]:
retweet_filter=' -filter:retweets' #this filters out retweets
tweetsPerQry = 100 # this is the max the API permits

In [25]:
def extract_user_name(user):
  return user["name"]

def get_tweets(searchQuery, maxTweets = 1000, sinceId = None, max_id = -1, 
               columns_of_interest = ['full_text', 'user', 'retweet_count'], filter_retweets = True):
  """Devuelve un dataframe con un determinado número de tweets dado por maxTweets que contienen el término de búsqueda searchQuery. 
  El argumento sinceId nos sirve para indicar un identificador de tweet desde el que empezar a descargar tweets, mientras que max_id
  nos da un valor máximo de identificador (a partir del cual no extraer tweets)."""
  df_final = pd.DataFrame(columns=columns_of_interest)
  tweetCount = 0
  if filter_retweets:
    searchQuery = searchQuery + retweet_filter
  while tweetCount < maxTweets:
    try:
        if (max_id <= 0):
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry, tweet_mode = 'extended')
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        since_id=sinceId, tweet_mode = 'extended')
        else:
            if (not sinceId):
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        max_id=str(max_id - 1), tweet_mode = 'extended')
            else:
                new_tweets = api.search(q=searchQuery, count=tweetsPerQry,
                                        max_id=str(max_id - 1),
                                        since_id=sinceId, tweet_mode = 'extended')
        if not new_tweets:
            print("No se han encontrado más tweets.")
            break
        #for tweet in new_tweets:
        #    f.write(jsonpickle.encode(tweet._json, unpicklable=False) +
        #            '\n')
        #Introducimos los tweets en el dataframe que devolveremos
        new_tweets_df = pd.DataFrame([tweet._json for tweet in new_tweets if tweet.lang=='en'])
        df_final = pd.concat([df_final, new_tweets_df[columns_of_interest]])

        tweetCount += new_tweets_df.shape[0]
        max_id = new_tweets[-1].id
    except tweepy.TweepError as e:
        # Just exit if any error
        print("Ha ocurrido algún error con la API de Twitter: " + str(e))
        break
  if df_final.shape[0] > maxTweets:
    df_final = df_final.head(maxTweets)
  print("Extraídos {0} tweets".format(min(tweetCount, maxTweets)))
  df_final.user = df_final.user.apply(lambda x: extract_user_name(x))
  return df_final

In [26]:
tweets = get_tweets("@westerndigital", maxTweets = 1000)

No se han encontrado más tweets.
Extraídos 118 tweets


In [27]:
tweets.user.iloc[0]

'Express Computer'

In [28]:
tweets.full_text.iloc[0]

'#TechSenate | The #education sector is an important area of focus for #WesternDigital: Manpreet Ahluwalia, Regional Sales Manager (North) &amp; Limton Xavier, Principal Engineer, @WesternDigital at the #DigitalTechnologySenate - #Education Track #TechnologySenate @srikrp'

In [29]:
tweets.head(3)

,full_text,user,retweet_count
0,#TechSenate | The #education sector is an impo...,Express Computer,0
1,#TechSenate | With universities using open sou...,Express Computer,0
2,#TechSenate | Our storage and camera products ...,Express Computer,0


Preprocesamos los tweets como hacíamos durante el entrenamiento (reemplazando las URLs por URL y los nombres de usuario por USER).

In [30]:
tweets = preprocesa_tweetsAPI_para_BERT(tweets)

Y finalmente ejecutamos nuestro clasificador de sentimientos en estos tweets.


In [31]:
sentimientos_df = pd.DataFrame(pipeline_BERT(list(tweets['full_text'].values)))
sentimientos_df.label = sentimientos_df.label.apply(lambda x: decode_labels(x))

In [32]:
sentimientos_df.label.value_counts()

1    66
0    52
Name: label, dtype: int64

Creamos ahora una función para automatizar todo este proceso.

In [33]:
def get_sentiment(searchQuery, maxTweets = 1000, sinceId = None, max_id = -1, 
               columns_of_interest = ['full_text', 'user', 'retweet_count'], filter_retweets = True):
  tweets = get_tweets(searchQuery, maxTweets = maxTweets, sinceId = sinceId, max_id = max_id, 
               columns_of_interest = columns_of_interest, filter_retweets = filter_retweets)
  tweets = preprocesa_tweetsAPI_para_BERT(tweets)
  sentimientos_df = pd.DataFrame(pipeline_BERT(list(tweets['full_text'].values)))
  sentimientos_df.label = sentimientos_df.label.apply(lambda x: decode_labels(x))
  return sentimientos_df

#### Sentimiento de tweets que mencionan personajes populares.
Por ejemplo, podemos usar nuestro clasificador para ver qué sentimiento tienen los últimos tweets que mencionan personajes de la escena política, como pueden ser Donald Trump o Joe Biden (líderes políticos de USA).


In [34]:
sentimientos_df = get_sentiment("@realDonaldTrump")

Extraídos 1000 tweets


In [35]:
sentimientos_df.label.value_counts()

1    652
0    348
Name: label, dtype: int64

In [36]:
sentimientos_df = get_sentiment("@JoeBiden")

Extraídos 1000 tweets


In [37]:
sentimientos_df.label.value_counts()

1    688
0    312
Name: label, dtype: int64

#### Sentimiento de tweets que mencionan empresas populares.
Podemos también usar nuestro clasificador para ver qué sentimiento tienen los últimos tweets que mencionan cuatro de las empresas técnológicas más populares del mundo.

In [38]:
sentimientos_df = get_sentiment("@Facebook")

Extraídos 1000 tweets


In [39]:
sentimientos_df.label.value_counts()

1    600
0    400
Name: label, dtype: int64

In [40]:
sentimientos_df = get_sentiment("@Google")

Extraídos 1000 tweets


In [41]:
sentimientos_df.label.value_counts()

1    672
0    328
Name: label, dtype: int64

In [42]:
sentimientos_df = get_sentiment("@amazon")

Extraídos 1000 tweets


In [43]:
sentimientos_df.label.value_counts()

1    736
0    264
Name: label, dtype: int64

In [44]:
sentimientos_df = get_sentiment("@Apple")

Extraídos 1000 tweets


In [45]:
sentimientos_df.label.value_counts()

1    553
0    447
Name: label, dtype: int64